In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'co2': 0.0008}
band = [4]
atmpro = saw
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band3b
  atmpro = mls
  band = 3b
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 0
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [5, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250)]
  tsfc = 294
  vmax = 720
  vmin = 620
  w_diffuse = [(1.66, 1.66, 1.66, 1.66, 1.66), (1.66, 1.66)]
  wgt = [(0, 0.6, 0.5, 0.7, 0.8), (0.8, 0.7)]


# Cooling Rate by Layer

CRD             CLIRAD `coolr_bands`           
            layer      coolrg                layer     coolrg
pressure                                                     
0.000312        1  103.125520                    1  61.101985
0.000750        2   35.419615                    2  43.823910
0.001052        3   33.861496                    3  42.492362
0.001476        4   30.438882                    4  38.731574
0.002070        5   26.874700                    5  32.954607
0.002904        6   23.796304                    6  26.420639
0.004074        7   21.332441                    7  20.857384
0.005714        8   19.417531                    8  17.584047
0.008015        9   17.909034                    9  16.488716
0.011243       10   16.647817                   10  16.522362
0.015771       11   15.510634                   11  16.644437
0.022122       12   14.457517                   12  16.277113
0.031031       13   13.651406                   13  15.383457
0.043528       14   12.573891                   14  13.590035
0.061057       15   10.976143                   15  11.088233
0.085645       16    9.627038                   16   9.189030
0.120136       17    8.816018                   17   8.478062
0.168516       18    8.385942                   18   8.578978
0.236378       19    8.261397                   19   8.963382
0.331549       20    8.452144                   20   9.366526
0.465100       21    9.295753                   21   9.975255
0.652400       22    8.866424                   22   8.955604
0.915100       23    6.860252                   23   6.352378
1.283650       24    5.254652                   24   4.638861
1.800600       25    4.193777                   25   4.098962
2.525700       26    3.372795                   26   3.704066
3.542800       27    2.646275                   27   2.957638
4.969550       28    2.128718                   28   2.138396
6.970850       29    1.881202                   29   1.852646
9.778100       30    1.664358                   30   1.881395
13.715850      31    1.378392                   31   1.727217
19.239350      32    1.031577                   32   1.213629
26.987250      33    0.806316                   33   0.793288
37.855300      34    0.705120                   34   0.736950
53.100050      35    0.552870                   35   0.676846
73.887500      36    0.401500                   36   0.460304
97.662500      37    0.302680                   37   0.252416
121.437500     38    0.214902                   38   0.131682
145.212500     39    0.143843                   39   0.074615
168.987500     40    0.106993                   40   0.063982
192.762500     41    0.080761                   41   0.059126
216.537500     42    0.059312                   42   0.052907
240.312500     43    0.038730                   43   0.043098
264.087500     44    0.006172                   44   0.018945
287.862500     45   -0.078333                   45  -0.065668
311.637500     46   -0.115157                   46  -0.104699
335.412500     47   -0.042357                   47  -0.033073
359.187500     48    0.015321                   48   0.019006
382.962500     49    0.029513                   49   0.027390
406.737500     50    0.034970                   50   0.030548
430.512500     51    0.037608                   51   0.032487
454.287500     52    0.038863                   52   0.032273
478.062500     53    0.039467                   53   0.032870
501.837500     54    0.039764                   54   0.032247
525.612500     55    0.040033                   55   0.031863
549.387500     56    0.040230                   56   0.031482
573.162500     57    0.040315                   57   0.031145
596.937500     58    0.040083                   58   0.030896
620.712500     59    0.039235                   59   0.029227
644.487500     60    0.037627                   60   0.028476
668.262500     61    0.035271                   61   0.025992
692.037500     62    0.032365                 

# Fluxes by Layer

CRD                       CLIRAD             \
                        flug       fldg      fnetg      i       flug   
pressure    level                                                      
0.000000    1     -14.101850   0.000000 -14.101850     11 -13.887722   
0.000624    2     -14.103465   0.009243 -14.094222     11 -13.889414   
0.000876    3     -14.104297   0.011130 -14.093167     11 -13.890146   
0.001229    4     -14.105313   0.013561 -14.091752     11 -13.891062   
0.001723    5     -14.106545   0.016577 -14.089968     11 -13.892198   
0.002417    6     -14.108031   0.020272 -14.087759     11 -13.893595   
0.003391    7     -14.109814   0.024799 -14.085015     11 -13.895298   
0.004757    8     -14.111943   0.030379 -14.081564     11 -13.897364   
0.006672    9     -14.114469   0.037311 -14.077158     11 -13.899849   
0.009359    10    -14.117442   0.045984 -14.071458     11 -13.902808   
0.013128    11    -14.120907   0.056881 -14.064025     11 -13.906282   
0.018415    12    -14.124883   0.070571 -14.054312     11 -13.910283   
0.025830    13    -14.129323   0.087712 -14.041612     11 -13.914740   
0.036232    14    -14.133847   0.109057 -14.024790     11 -13.919275   
0.050823    15    -14.138055   0.134997 -14.003057     11 -13.923517   
0.071291    16    -14.142183   0.165737 -13.976446     11 -13.927668   
0.100000    17    -14.146412   0.202706 -13.943706     11 -13.931854   
0.140271    18    -14.150337   0.248687 -13.901650     11 -13.935600   
0.196760    19    -14.153053   0.307518 -13.845535     11 -13.937877   
0.275997    20    -14.152582   0.384590 -13.767992     11 -13.936539   
0.387100    21    -14.143868   0.487114 -13.656754     11 -13.926356   
0.543100    22    -14.108504   0.623529 -13.484975     11 -13.889971   
0.761700    23    -14.035971   0.780590 -13.255381     11 -13.818236   
1.068500    24    -13.942643   0.936582 -13.006062     11 -13.726341   
1.498800    25    -13.836882   1.098661 -12.738221     11 -13.621748   
2.102400    26    -13.717747   1.279384 -12.438363     11 -13.502402   
2.949000    27    -13.586707   1.486587 -12.100120     11 -13.370219   
4.136600    28    -13.457503   1.729661 -11.727843     11 -13.237891   
5.802500    29    -13.343840   2.036074 -11.307766     11 -13.119258   
8.139200    30    -13.235225   2.448173 -10.787052     11 -13.002933   
11.417000   31    -13.124217   2.983399 -10.140817     11 -12.882627   
16.014700   32    -13.030225   3.640122  -9.390103     11 -12.780378   
22.464000   33    -13.024134   4.422119  -8.602014     11 -12.771740   
31.510500   34    -13.141150   5.403201  -7.737948     11 -12.899465   
44.200100   35    -13.315667   6.637637  -6.678030     11 -13.090996   
62.000000   36    -13.516806   8.004516  -5.512290     11 -13.308966   
85.775000   37    -13.692998   9.311460  -4.381538     11 -13.500707   
109.550000  38    -13.768439  10.239343  -3.529096     11 -13.582124   
133.325000  39    -13.788262  10.864399  -2.923863     11 -13.600819   
157.100000  40    -13.810476  11.291720  -2.518755     11 -13.621590   
180.875000  41    -13.838915  11.621486  -2.217429     11 -13.648336   
204.650000  42    -13.875805  11.885825  -1.989980     11 -13.682706   
228.425000  43    -13.924950  12.102011  -1.822939     11 -13.727293   
252.200000  44    -13.995075  12.281210  -1.713865     11 -13.786901   
275.975000  45    -14.128940  12.432458  -1.696482     11 -13.899939   
299.750000  46    -14.509701  12.592609  -1.917092     11 -14.268339   
323.525000  47    -15.203965  12.962554  -2.241411     11 -14.973311   
347.300000  48    -16.019876  13.659175  -2.360701     11 -15.810658   
371.075000  49    -16.834788  14.517235  -2.317552     11 -16.646023   
394.850000  50    -17.637429  15.402994  -2.234435     11 -17.467295   
418.625000  51    -18.426222  16.290274  -2.135948     11 -18.272880   
442.400000  52    -19.199758  17.169726  -2.030032     11 -19.061065   
466.175000  53    -19.956219  18.035638  -1.920581     11 -19.832

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')